# **BirdCLEF 2025 Training Notebook**

This is a baseline training pipeline for BirdCLEF 2025 using EfficientNetB0 with PyTorch and Timm(for pretrained EffNet). You can check inference and preprocessing notebooks in the following links: 

- [EfficientNet B0 Pytorch [Inference] | BirdCLEF'25](https://www.kaggle.com/code/kadircandrisolu/efficientnet-b0-pytorch-inference-birdclef-25)

  
- [Transforming Audio-to-Mel Spec. | BirdCLEF'25](https://www.kaggle.com/code/kadircandrisolu/transforming-audio-to-mel-spec-birdclef-25)  

Note that by default this notebook is in Debug Mode, so it will only train the model with 2 epochs, but the [weight](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-effnetb0-starter-weight) I used in the inference notebook was obtained after 10 epochs of training.

**Features**
* Implement with Pytorch and Timm
* Flexible audio processing with both pre-computed and on-the-fly mel spectrograms
* Stratified 5-fold cross-validation with ensemble capability
* Mixup training for improved generalization
* Spectrogram augmentations (time/frequency masking, brightness adjustment)
* AdamW optimizer with Cosine Annealing LR scheduling
* Debug mode for quick experimentation with smaller datasets

**Pre-computed Spectrograms**
For faster training, you can use pre-computed mel spectrograms from [this dataset](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-mel-spectrograms) by setting `LOAD_DATA = True`

## Libraries

In [1]:
# === ライブラリのインポート ===
# 基本的なライブラリ
import os  # OS操作（ファイルパスなど）
import logging  # ログ出力用
import random  # 乱数生成
import gc  # ガベージコレクション（メモリ管理）
import time  # 時間計測
import cv2  # OpenCV（画像処理、ここではリサイズに使用）
import math  # 数学演算
import warnings  # 警告メッセージの制御
from pathlib import Path  # オブジェクト指向のファイルパス操作

# データ操作・数値計算
import numpy as np  # 数値計算（配列操作）
import pandas as pd  # データフレーム操作

# 機械学習・評価
from sklearn.model_selection import StratifiedKFold  # 層化K分割交差検証
from sklearn.metrics import roc_auc_score  # ROC AUCスコア計算

# 音声処理
import librosa  # 音声分析ライブラリ (メルスペクトログラム計算など)

# 深層学習 (PyTorch)
import torch  # PyTorch本体
import torch.nn as nn  # ニューラルネットワークのモジュール（層、損失関数など）
import torch.nn.functional as F  # よく使う関数（活性化関数など）
import torch.optim as optim  # 最適化アルゴリズム (Adam, SGDなど)
from torch.optim import lr_scheduler  # 学習率スケジューラ
from torch.utils.data import Dataset, DataLoader  # データセット、データローダー

# 可視化
import matplotlib.pyplot as plt  # グラフ描画
import seaborn as sns  # 統計的グラフ描画

# ユーティリティ
from tqdm.auto import tqdm  # プログレスバー表示
import timm  # PyTorch Image Models (事前学習済みモデルライブラリ) <= ★これ便利！

# 警告を非表示に設定
warnings.filterwarnings("ignore")
# ログレベルをERROR以上に設定（INFOやWARNINGを表示しない）
logging.basicConfig(level=logging.ERROR)

## Configuration

In [2]:
# === 設定クラス ===
# 学習やデータに関する様々なパラメータをまとめて管理するためのクラス
class CFG:
    # --- 基本設定 ---
    seed = 42  # 乱数シード（結果の再現性のため）
    debug = True  # デバッグモードフラグ (Trueだと少量データで短時間実行)
    apex = False # NVIDIA Apexを使った混合精度演算を使うか (今回はFalse)
    print_freq = 100 # ログ出力の頻度（ステップ数）
    num_workers = 2 # データローダーで使うCPUプロセスの数 (並列処理)

    # --- パス設定 ---
    OUTPUT_DIR = '/kaggle/working/' # 学習済みモデルなどの出力先ディレクトリ

    # 入力データパス (Kaggle環境のパス)
    train_datadir = '/kaggle/input/birdclef-2025/train_audio' # 学習用音声ファイルディレクトリ
    train_csv = '/kaggle/input/birdclef-2025/train.csv' # 学習用メタデータCSV
    test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes' # テスト用音声ファイルディレクトリ (Soundscape)
    submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv' # 提出フォーマットファイル
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv' # 鳥の分類情報CSV (クラスラベル取得用)

    # 事前計算済みスペクトログラムのパス (NPYファイル)
    spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'

    # --- モデル設定 ---
    model_name = 'efficientnet_b0'  # 使用するモデル名 (timmライブラリに対応) <= ★EfficientNet-B0を採用
    pretrained = True # 事前学習済み重みを使用するか
    in_channels = 1 # 入力チャネル数 (メルスペクトログラムはグレースケールなので1)

    # --- データ処理設定 ---
    LOAD_DATA = True  # 事前計算済みのスペクトログラム(.npy)をロードするかどうか <= ★Trueだと高速
    FS = 32000 # サンプリング周波数 (Hz)
    TARGET_DURATION = 5.0 # 音声クリップの目標時間 (秒)
    TARGET_SHAPE = (256, 256) # メルスペクトログラムの目標形状 (高さ, 幅)

    # メルスペクトログラム計算パラメータ (librosa用)
    N_FFT = 1024 # FFTの窓サイズ
    HOP_LENGTH = 512 # ホップ長（窓をスライドさせる幅）
    N_MELS = 128 # メルフィルタバンクの数（スペクトログラムの高さ次元）
    FMIN = 50 # 考慮する最小周波数 (Hz)
    FMAX = 14000 # 考慮する最大周波数 (Hz)

    # --- 学習設定 ---
    device = 'cuda' if torch.cuda.is_available() else 'cpu' # 使用デバイス (GPUがあれば'cuda')
    epochs = 10  # 学習エポック数 (debug=Trueだと後で上書きされる)
    batch_size = 32  # バッチサイズ
    criterion = 'BCEWithLogitsLoss' # 損失関数名 (マルチラベル分類なのでBCEWithLogitsLoss)

    # --- 交差検証 (Cross-Validation) 設定 ---
    n_fold = 5 # 分割数 (5-fold CV)
    selected_folds = [0, 1, 2, 3, 4] # 実行するFoldのリスト (debug=Trueだと後で上書きされる)

    # --- 最適化 (Optimizer) 設定 ---
    optimizer = 'AdamW' # 最適化アルゴリズム名 <= ★AdamWを採用
    lr = 5e-4 # 学習率 (Learning Rate)
    weight_decay = 1e-5 # 重み減衰 (L2正則化)

    # --- 学習率スケジューラ設定 ---
    scheduler = 'CosineAnnealingLR' # スケジューラ名 <= ★CosineAnnealingLRを採用
    min_lr = 1e-6 # 最小学習率 (CosineAnnealingLR用)
    T_max = epochs # CosineAnnealingLRの1サイクルのエポック数

    # --- データ拡張・Mixup設定 ---
    aug_prob = 0.5  # データ拡張を適用する確率
    mixup_alpha = 0.5  # Mixupのアルファ値 (0より大きい場合Mixup有効) <= ★Mixupを採用

    # デバッグモード時の設定を上書きするメソッド
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 10 # エポック数を2に
            self.selected_folds = [0] # Fold 0 のみ実行

# CFGクラスのインスタンスを作成
cfg = CFG()

## Utilities

In [3]:
# === ユーティリティ関数 ===

def set_seed(seed=42):
    """
    再現性のために乱数シードを固定する関数
    """
    # Python標準のrandomモジュール
    random.seed(seed)
    # ハッシュのシード (辞書のキー順序などに関わる)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # NumPyの乱数シード
    np.random.seed(seed)
    # PyTorchのCPU乱数シード
    torch.manual_seed(seed)
    # PyTorchの現在のGPU乱数シード
    torch.cuda.manual_seed(seed)
    # PyTorchの全GPU乱数シード (複数のGPUがある場合)
    torch.cuda.manual_seed_all(seed)
    # cuDNNの決定論的アルゴリズムを使用する設定 (Trueにすると再現性が上がるが遅くなる可能性)
    torch.backends.cudnn.deterministic = True
    # cuDNNのベンチマークモードを無効化 (Trueだと最適なアルゴリズムを探すが結果が変わる可能性)
    torch.backends.cudnn.benchmark = False

# シードを固定
set_seed(cfg.seed)

## Pre-processing
These functions handle the transformation of audio files to mel spectrograms for model input, with flexibility controlled by the `LOAD_DATA` parameter. The process involves either loading pre-computed spectrograms from this [dataset](https://www.kaggle.com/datasets/kadircandrisolu/birdclef25-mel-spectrograms) (when `LOAD_DATA=True`) or dynamically generating them (when `LOAD_DATA=False`), transforming audio data into spectrogram representations, and preparing it for the neural network.

In [4]:
# === 前処理関数 ===
# 音声データからメルスペクトログラムを生成する関数群

def audio2melspec(audio_data, cfg):
    """
    音声データ (NumPy配列) をメルスペクトログラムに変換する関数
    """
    # もしデータにNaNが含まれていたら、平均値で補完 (念のため)
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    # librosaを使ってメルスペクトログラムを計算
    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,         # 入力音声データ
        sr=cfg.FS,            # サンプリング周波数
        n_fft=cfg.N_FFT,      # FFT窓サイズ
        hop_length=cfg.HOP_LENGTH, # ホップ長
        n_mels=cfg.N_MELS,    # メルフィルタバンク数
        fmin=cfg.FMIN,        # 最小周波数
        fmax=cfg.FMAX,        # 最大周波数
        power=2.0             # パワースペクトログラム (**2) を計算
    )

    # パワースペクトログラムをデシベル(dB)スケールに変換
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    # デシベルスケールのスペクトログラムを0-1の範囲に正規化
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8) # ゼロ除算防止

    return mel_spec_norm

def process_audio_file(audio_path, cfg):
    """
    単一の音声ファイルを処理してメルスペクトログラムを取得する関数
    (LOAD_DATA=False の場合、Datasetクラスから呼ばれる)
    """
    try:
        # librosaで音声ファイルをロード (指定サンプリング周波数でリサンプリングされる)
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)

        # 目標サンプル数を計算 (目標時間 * サンプリング周波数)
        target_samples = int(cfg.TARGET_DURATION * cfg.FS)

        # 音声が目標サンプル数より短い場合、繰り返して長さを合わせる
        if len(audio_data) < target_samples:
            n_copy = math.ceil(target_samples / len(audio_data)) # 必要な繰り返し回数
            if n_copy > 1:
                audio_data = np.concatenate([audio_data] * n_copy) # 配列を指定回数連結

        # 音声の中央から目標サンプル数分を抽出 (5秒クリップ)
        start_idx = max(0, int(len(audio_data) / 2 - target_samples / 2)) # 開始インデックス
        end_idx = min(len(audio_data), start_idx + target_samples) # 終了インデックス
        center_audio = audio_data[start_idx:end_idx] # 中央部分をスライス

        # 抽出後、長さが足りない場合 (ほぼ起こらないはずだが念のため)、ゼロパディング
        if len(center_audio) < target_samples:
            center_audio = np.pad(center_audio,
                                 (0, target_samples - len(center_audio)),
                                 mode='constant') # 末尾を0で埋める

        # 中央5秒の音声データをメルスペクトログラムに変換
        mel_spec = audio2melspec(center_audio, cfg)

        # スペクトログラムの形状が目標形状と異なる場合、リサイズ
        # (librosaの計算結果は厳密には目標形状にならないことがあるため)
        if mel_spec.shape != cfg.TARGET_SHAPE:
            mel_spec = cv2.resize(mel_spec, cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR) # 線形補間でリサイズ

        # float32形式で返す (PyTorchのTensorにしやすい)
        return mel_spec.astype(np.float32)

    except Exception as e:
        # エラーが発生した場合、ログを出力してNoneを返す
        print(f"Error processing {audio_path}: {e}")
        return None

def generate_spectrograms(df, cfg):
    """
    データフレーム内の全音声ファイルからメルスペクトログラムを生成する関数
    (このノートブックでは LOAD_DATA=True のため、実際にはあまり使われない)
    """
    print("Generating mel spectrograms from audio files...")
    start_time = time.time() # 開始時間記録

    all_bird_data = {} # 結果を格納する辞書 {samplename: spectrogram}
    errors = [] # エラーが発生したファイル情報を格納するリスト

    # データフレームを行ごとに処理 (tqdmでプログレスバー表示)
    for i, row in tqdm(df.iterrows(), total=len(df)):
        # デバッグモードの場合、1000件処理したら終了
        if cfg.debug and i >= 1000:
            break

        try:
            samplename = row['samplename'] # サンプル名取得
            filepath = row['filepath'] # ファイルパス取得

            # 音声ファイルを処理してスペクトログラムを取得
            mel_spec = process_audio_file(filepath, cfg)

            # 処理が成功したら辞書に格納
            if mel_spec is not None:
                all_bird_data[samplename] = mel_spec

        except Exception as e:
            # 予期せぬエラーをキャッチ
            print(f"Error processing {row.filepath}: {e}")
            errors.append((row.filepath, str(e)))

    end_time = time.time() # 終了時間記録
    # 処理結果のサマリを出力
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    print(f"Successfully processed {len(all_bird_data)} files out of {len(df)}")
    print(f"Failed to process {len(errors)} files")

    return all_bird_data # 生成したスペクトログラムの辞書を返す

## Dataset Preparation and Data Augmentations
We'll convert audio to mel spectrograms and apply random augmentations with 50% probability each - including time stretching, pitch shifting, and volume adjustments. This randomized approach creates diverse training samples from the same audio files

In [5]:
# === PyTorch Datasetクラス ===
# 音声データ (または事前計算スペクトログラム) とラベルをモデルに適した形式で提供するクラス

class BirdCLEFDatasetFromNPY(Dataset):
    def __init__(self, df, cfg, spectrograms=None, mode="train"):
        """
        コンストラクタ
        Args:
            df (pd.DataFrame): データフレーム (train_df または val_df)
            cfg (CFG): 設定オブジェクト
            spectrograms (dict, optional): 事前計算されたスペクトログラムの辞書. Defaults to None.
            mode (str, optional): "train" または "valid". Defaults to "train".
        """
        self.df = df # データフレームを保持
        self.cfg = cfg # 設定オブジェクトを保持
        self.mode = mode # モード ('train' or 'valid') を保持

        # 事前計算されたスペクトログラムの辞書を保持
        self.spectrograms = spectrograms

        # 分類情報CSVを読み込み、クラス名リストとラベル->インデックス辞書を作成
        taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        self.species_ids = taxonomy_df['primary_label'].tolist() # 全クラス名のリスト
        self.num_classes = len(self.species_ids) # クラス総数
        self.label_to_idx = {label: idx for idx, label in enumerate(self.species_ids)} # ラベル名をインデックスに変換する辞書

        # データフレームに 'filepath' 列がなければ作成 (元CSVにはfilenameしかない場合があるため)
        if 'filepath' not in self.df.columns:
            self.df['filepath'] = self.cfg.train_datadir + '/' + self.df.filename
        # データフレームに 'samplename' 列がなければ作成 (NPYのキーと合わせるため)
        if 'samplename' not in self.df.columns:
            #例: 'asbfly/XC134896.ogg' -> 'asbfly-XC134896'
            self.df['samplename'] = self.df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])

        # このデータセットに含まれるべきサンプル名
        sample_names = set(self.df['samplename'])
        # もし事前計算スペクトログラムがあれば、何件見つかったか表示
        if self.spectrograms:
            found_samples = sum(1 for name in sample_names if name in self.spectrograms)
            print(f"Found {found_samples} matching spectrograms for {mode} dataset out of {len(self.df)} samples")

        # デバッグモードの場合、データ数を1000件以下にサンプリング
        if cfg.debug:
            self.df = self.df.sample(min(1000, len(self.df)), random_state=cfg.seed).reset_index(drop=True)

    def __len__(self):
        """ データセットのサンプル数を返す """
        return len(self.df)

    def __getitem__(self, idx):
        """
        指定されたインデックス `idx` のサンプル（スペクトログラムとターゲット）を取得するメソッド
        """
        # データフレームから該当行を取得
        row = self.df.iloc[idx]
        samplename = row['samplename'] # サンプル名を取得
        spec = None # スペクトログラム初期化

        # 1. 事前計算スペクトログラムの辞書があり、かつサンプル名が存在すれば、そこから取得
        if self.spectrograms and samplename in self.spectrograms:
            spec = self.spectrograms[samplename]
        # 2. 事前計算を使わない設定 (LOAD_DATA=False) の場合、オンデマンドで生成
        elif not self.cfg.LOAD_DATA:
            spec = process_audio_file(row['filepath'], self.cfg)

        # スペクトログラムが取得/生成できなかった場合 (エラーなど)
        if spec is None:
            # ターゲット形状のゼロ埋め配列を返す
            spec = np.zeros(self.cfg.TARGET_SHAPE, dtype=np.float32)
            # 学習時のみ警告を表示
            if self.mode == "train":
                print(f"Warning: Spectrogram for {samplename} not found and could not be generated")

        # NumPy配列をPyTorch Tensorに変換し、チャンネル次元を追加 (H, W) -> (C=1, H, W)
        spec = torch.tensor(spec, dtype=torch.float32).unsqueeze(0)

        # 学習モードかつランダムな確率でデータ拡張を適用
        if self.mode == "train" and random.random() < self.cfg.aug_prob:
            spec = self.apply_spec_augmentations(spec)

        # ラベルをエンコード (Multi-hotベクトルに)
        target = self.encode_label(row['primary_label'])

        # secondary_labels が存在し、空でない場合、それらもターゲットに追加
        if 'secondary_labels' in row and row['secondary_labels'] not in [[''], None, np.nan]:
            # 文字列形式 ('['label1', 'label2']') の場合、評価してリストに変換
            if isinstance(row['secondary_labels'], str):
                try: # 安全のためtry-except
                    secondary_labels = eval(row['secondary_labels'])
                except:
                    secondary_labels = []
            else: # すでにリスト形式の場合
                secondary_labels = row['secondary_labels']

            # 各セカンダリラベルに対応するインデックスを1にする
            for label in secondary_labels:
                if label in self.label_to_idx:
                    target[self.label_to_idx[label]] = 1.0

        # 辞書形式でスペクトログラム、ターゲット、ファイル名を返す
        return {
            'melspec': spec,
            'target': torch.tensor(target, dtype=torch.float32),
            'filename': row['filename'] # ファイル名も返す（デバッグや分析用）
        }

    def apply_spec_augmentations(self, spec):
        """ スペクトログラムにデータ拡張を適用する関数 """

        # 時間マスキング (スペクトログラムの縦方向の一部を隠す)
        if random.random() < 0.5: # 50%の確率で適用
            num_masks = random.randint(1, 3) # 1〜3個のマスクを適用
            for _ in range(num_masks):
                width = random.randint(5, 20) # マスクの幅（時間ステップ数）
                start = random.randint(0, spec.shape[2] - width) # マスク開始位置
                spec[0, :, start:start+width] = 0 # マスク部分を0で埋める

        # 周波数マスキング (スペクトログラムの横方向の一部を隠す)
        if random.random() < 0.5: # 50%の確率で適用
            num_masks = random.randint(1, 3) # 1〜3個のマスクを適用
            for _ in range(num_masks):
                height = random.randint(5, 20) # マスクの高さ（周波数ビン数）
                start = random.randint(0, spec.shape[1] - height) # マスク開始位置
                spec[0, start:start+height, :] = 0 # マスク部分を0で埋める

        # ランダムな明るさ/コントラスト調整
        if random.random() < 0.5: # 50%の確率で適用
            gain = random.uniform(0.8, 1.2) # ゲイン（コントラスト）をランダムに
            bias = random.uniform(-0.1, 0.1) # バイアス（明るさ）をランダムに
            spec = spec * gain + bias # 適用
            spec = torch.clamp(spec, 0, 1) # 値を0-1の範囲にクリップ

        return spec

    def encode_label(self, label):
        """ ラベル文字列を Multi-hot ベクトルにエンコードする関数 """
        # クラス数分のゼロベクトルを作成
        target = np.zeros(self.num_classes)
        # ラベルが既知のクラスなら、対応するインデックスを1.0にする
        if label in self.label_to_idx:
            target[self.label_to_idx[label]] = 1.0
        return target

In [6]:
# === カスタム Collate 関数 ===
# DataLoaderがバッチを作成する際に、サンプルリストをどのようにまとめるかを定義する関数

def collate_fn(batch):
    """
    バッチ内のサンプルを適切にまとめるカスタムcollate関数
    (Datasetの__getitem__がNoneを返す可能性を考慮しているが、現状はゼロ埋めのため、
     主にテンソルのスタックが目的)
    """
    # バッチからNoneの要素を除去 (エラーでNoneが返された場合などに対応)
    batch = [item for item in batch if item is not None]
    # もしバッチが空になったら空の辞書を返す
    if len(batch) == 0:
        return {}

    # バッチ内の最初のサンプルのキーを使って結果用の辞書を初期化
    result = {key: [] for key in batch[0].keys()}

    # 各サンプルの値を対応するキーのリストに追加
    for item in batch:
        for key, value in item.items():
            result[key].append(value)

    # キーごとに値を適切に処理
    for key in result:
        # 'target' キーの値がTensorなら、リスト内のTensorをスタックしてバッチ次元を作る
        if key == 'target' and isinstance(result[key][0], torch.Tensor):
            result[key] = torch.stack(result[key])
        # 'melspec' キーの値がTensorなら、同様にスタック
        elif key == 'melspec' and isinstance(result[key][0], torch.Tensor):
            # (念のため) バッチ内の全スペクトログラムの形状が同じか確認してからスタック
            shapes = [t.shape for t in result[key]]
            if len(set(str(s) for s in shapes)) == 1: # 全形状が同じなら
                result[key] = torch.stack(result[key])
            # もし形状が異なっていたら... (現状のリサイズ処理では起こりにくいはず)
            # ここではリストのまま返すか、エラーを出すか等の処理が必要になるが、
            # このコードでは形状が同じ前提でスタックのみ行っている

    return result # まとめたバッチ(辞書)を返す

## Model Definition

In [7]:
# === モデル定義 ===
# BirdCLEFタスク用のPyTorchモデルクラス

class BirdCLEFModel(nn.Module):
    def __init__(self, cfg):
        """コンストラクタ"""
        super().__init__() # 親クラス(nn.Module)のコンストラクタ呼び出し
        self.cfg = cfg # 設定オブジェクトを保持

        # 分類情報CSVからクラス数を取得
        taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
        cfg.num_classes = len(taxonomy_df) # クラス数をcfgに追加（または上書き）

        # timmライブラリを使って事前学習済みモデルをロード
        self.backbone = timm.create_model(
            cfg.model_name,           # モデル名 (例: 'efficientnet_b0')
            pretrained=cfg.pretrained,# 事前学習済み重みを使うか
            in_chans=cfg.in_channels, # 入力チャネル数 (1 = グレースケール)
            drop_rate=0.2,            # ドロップアウト率 (全結合層の手前)
            drop_path_rate=0.2        # DropPath率 (Stochastic Depth)
        )

        # モデルの種類に応じて、最終層(分類層)を削除し、その入力特徴量を保持
        if 'efficientnet' in cfg.model_name:
            # EfficientNetの場合、classifier層の入力特徴量数を取得
            backbone_out = self.backbone.classifier.in_features
            # 元のclassifier層をIdentity（何もしない層）に置き換え
            self.backbone.classifier = nn.Identity()
        elif 'resnet' in cfg.model_name:
            # ResNet系の場合、fc層の入力特徴量数を取得
            backbone_out = self.backbone.fc.in_features
            # 元のfc層をIdentityに置き換え
            self.backbone.fc = nn.Identity()
        else: # その他のtimmモデルの場合 (汎用的な取得方法)
            backbone_out = self.backbone.get_classifier().in_features
            # 元の分類層をリセット (クラス数を0に)
            self.backbone.reset_classifier(0, '')

        # Global Average Pooling層 (特徴マップをベクトルに変換)
        self.pooling = nn.AdaptiveAvgPool2d(1)

        # 特徴ベクトルの次元数
        self.feat_dim = backbone_out

        # 新しい分類層 (バックボーンからの特徴量を入力とし、クラス数を出力)
        self.classifier = nn.Linear(backbone_out, cfg.num_classes)

        # Mixupが設定で有効になっているか確認
        self.mixup_enabled = hasattr(cfg, 'mixup_alpha') and cfg.mixup_alpha > 0
        if self.mixup_enabled:
            # Mixupのアルファ値を保持
            self.mixup_alpha = cfg.mixup_alpha

    def forward(self, x, targets=None):
        """
        順伝播メソッド
        Args:
            x (torch.Tensor): 入力テンソル (バッチサイズ, 1, 高さ, 幅)
            targets (torch.Tensor, optional): ターゲットテンソル (Mixup用). Defaults to None.
        Returns:
            torch.Tensor: モデルの出力ロジット (バッチサイズ, クラス数)
            or tuple(torch.Tensor, torch.Tensor): Mixup有効な訓練時は (ロジット, Mixup損失)
        """

        # --- Mixup処理 (学習時かつMixup有効の場合) ---
        if self.training and self.mixup_enabled and targets is not None:
            # mixup_dataメソッドで入力とターゲットをミックス
            mixed_x, targets_a, targets_b, lam = self.mixup_data(x, targets)
            x = mixed_x # 入力をミックスしたものに置き換え
        else:
            # Mixupしない場合、関連変数をNoneに
            targets_a, targets_b, lam = None, None, None

        # --- 特徴抽出 ---
        # バックボーンモデルに入力を通して特徴量を取得
        features = self.backbone(x)

        # 一部のtimmモデルは辞書形式で特徴量を返すことがあるため対応
        if isinstance(features, dict):
            features = features['features'] # 想定されるキーから取得

        # 特徴量が4次元テンソル (B, C, H, W) の場合、プーリングしてベクトル化
        if len(features.shape) == 4:
            features = self.pooling(features) # (B, C, 1, 1)
            features = features.view(features.size(0), -1) # (B, C) に変形

        # --- 分類 ---
        # 最終的な分類層（線形層）に入力し、ロジット（活性化関数適用前の出力）を得る
        logits = self.classifier(features)

        # --- Mixup損失計算 (学習時かつMixup有効の場合) ---
        if self.training and self.mixup_enabled and targets is not None:
            # mixup_criterionメソッドでミックスされたターゲットに対する損失を計算
            loss = self.mixup_criterion(F.binary_cross_entropy_with_logits,
                                       logits, targets_a, targets_b, lam)
            # ロジットと計算済み損失のタプルを返す
            return logits, loss

        # 通常時 (Mixupなし、または推論時) はロジットのみを返す
        return logits

    def mixup_data(self, x, targets):
        """ Mixupを実行するメソッド """
        batch_size = x.size(0) # バッチサイズ取得

        # ベータ分布からミックス比率ラムダ(lam)をサンプリング
        lam = np.random.beta(self.mixup_alpha, self.mixup_alpha)

        # バッチ内でシャッフルするためのインデックスを作成
        indices = torch.randperm(batch_size).to(x.device)

        # 入力データをミックス (x = lam * x_a + (1 - lam) * x_b)
        mixed_x = lam * x + (1 - lam) * x[indices]
        # 対応するターゲットも取得 (ミックスはしない、それぞれのターゲットを保持)
        targets_a, targets_b = targets, targets[indices]

        return mixed_x, targets_a, targets_b, lam

    def mixup_criterion(self, criterion, pred, y_a, y_b, lam):
        """ Mixup時の損失関数を計算するメソッド """
        # 損失 = lam * Loss(pred, y_a) + (1 - lam) * Loss(pred, y_b)
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## Training Utilities
We are configuring our optimization strategy with the AdamW optimizer, cosine scheduling, and the BCEWithLogitsLoss criterion.

In [8]:
# === 学習用ユーティリティ関数 ===
# オプティマイザ、スケジューラ、損失関数を生成するファクトリ関数

def get_optimizer(model, cfg):
    """ 設定に基づいてオプティマイザを生成する """
    if cfg.optimizer == 'Adam':
        optimizer = optim.Adam(
            model.parameters(), # 最適化対象のモデルパラメータ
            lr=cfg.lr,          # 学習率
            weight_decay=cfg.weight_decay # 重み減衰
        )
    elif cfg.optimizer == 'AdamW': # <= 今回はこれ
        optimizer = optim.AdamW(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'SGD':
        optimizer = optim.SGD(
            model.parameters(),
            lr=cfg.lr,
            momentum=0.9, # モーメンタム
            weight_decay=cfg.weight_decay
        )
    else: # 未対応のオプティマイザ名の場合
        raise NotImplementedError(f"Optimizer {cfg.optimizer} not implemented")

    return optimizer

def get_scheduler(optimizer, cfg):
    """ 設定に基づいて学習率スケジューラを生成する """
    if cfg.scheduler == 'CosineAnnealingLR': # <= 今回はこれ
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer,      # 対象のオプティマイザ
            T_max=cfg.T_max,  # 1サイクルのエポック数
            eta_min=cfg.min_lr # 最小学習率
        )
    elif cfg.scheduler == 'ReduceLROnPlateau': # 検証ロスが改善しなくなったらLRを下げる
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',     # 'min' (ロス最小化) or 'max' (精度最大化)
            factor=0.5,     # 学習率を減らす係数 (new_lr = lr * factor)
            patience=2,     # 何エポック改善しなかったら発動するか
            min_lr=cfg.min_lr,# 最小学習率
            verbose=True    # 発動時にメッセージ表示
        )
    elif cfg.scheduler == 'StepLR': # 指定エポックごとにLRを下げる
        scheduler = lr_scheduler.StepLR(
            optimizer,
            step_size=cfg.epochs // 3, # 何エポックごとにLRを下げるか
            gamma=0.5          # 学習率を減らす係数 (new_lr = lr * gamma)
        )
    elif cfg.scheduler == 'OneCycleLR': # OneCycleLR (別の場所で設定される想定)
        scheduler = None # ここではNoneを返す
    else: # スケジューラなし、または未対応の場合
        scheduler = None

    return scheduler

def get_criterion(cfg):
    """ 設定に基づいて損失関数を生成する """
    if cfg.criterion == 'BCEWithLogitsLoss': # <= 今回はこれ
        # マルチラベル分類用のバイナリクロスエントロピー損失 (Sigmoid + BCE)
        # モデルの出力はロジット (活性化関数適用前) である必要がある
        criterion = nn.BCEWithLogitsLoss()
    else: # 未対応の損失関数名の場合
        raise NotImplementedError(f"Criterion {cfg.criterion} not implemented")

    return criterion

## Training Loop

In [9]:
# === 学習・検証ループ関数 ===

def train_one_epoch(model, loader, optimizer, criterion, device, scheduler=None):
    """ 1エポック分の学習処理を行う関数 """

    model.train() # モデルを学習モードに設定 (Dropoutなどが有効になる)
    losses = [] # 各バッチの損失を格納するリスト
    all_targets = [] # 全ターゲットラベルを格納するリスト
    all_outputs = [] # 全モデル出力を格納するリスト

    # データローダーからバッチを取得してループ (tqdmでプログレスバー表示)
    pbar = tqdm(enumerate(loader), total=len(loader), desc="Training")

    for step, batch in pbar:

        # --- バッチ処理 (CollateFnがリストを返す可能性への対応 - 若干冗長かも) ---
        # もしバッチの'melspec'がリスト形式だったら (通常はTensorのはず)
        if isinstance(batch['melspec'], list):
            # (この部分は現状のCollateFnではあまり通らない想定だが、念のため)
            batch_outputs = []
            batch_losses = []

            # リスト内の各サンプルを個別に処理
            for i in range(len(batch['melspec'])):
                # 1サンプル分の入力とターゲットを取得し、デバイスに転送
                inputs = batch['melspec'][i].unsqueeze(0).to(device)
                target = batch['target'][i].unsqueeze(0).to(device)

                optimizer.zero_grad() # 勾配を初期化
                output = model(inputs) # 順伝播
                loss = criterion(output, target) # 損失計算
                loss.backward() # 誤差逆伝播

                batch_outputs.append(output.detach().cpu()) # 出力をCPUに戻して保存
                batch_losses.append(loss.item()) # 損失値を保存

            optimizer.step() # パラメータ更新 (全サンプル処理後)
            outputs = torch.cat(batch_outputs, dim=0).numpy() # 出力を結合
            loss = np.mean(batch_losses) # 損失の平均値
            targets = batch['target'].numpy() # ターゲット (すでにCPUにある想定)

        # --- 通常のバッチ処理 (入力がTensorの場合) ---
        else:
            # 入力とターゲットをデバイスに転送
            inputs = batch['melspec'].to(device)
            targets = batch['target'].to(device)

            optimizer.zero_grad() # 勾配を初期化
            # 順伝播 (Mixup有効時は (logits, loss) のタプルが返る)
            outputs = model(inputs, targets if model.training and model.mixup_enabled else None)

            # Mixup有効時 (タプルが返ってくる)
            if isinstance(outputs, tuple):
                outputs, loss = outputs # ロジットと損失をアンパック
            # Mixup無効時 (ロジットのみ返ってくる)
            else:
                loss = criterion(outputs, targets) # 損失を計算

            loss.backward() # 誤差逆伝播
            optimizer.step() # パラメータ更新

            # 結果をCPUに戻してNumPy配列に変換
            outputs = outputs.detach().cpu().numpy()
            targets = targets.detach().cpu().numpy()

        # OneCycleLRスケジューラの場合、ステップごとに更新
        if scheduler is not None and isinstance(scheduler, lr_scheduler.OneCycleLR):
            scheduler.step()

        # --- 結果の保存 ---
        all_outputs.append(outputs) # モデル出力をリストに追加
        all_targets.append(targets) # ターゲットラベルをリストに追加
        losses.append(loss if isinstance(loss, float) else loss.item()) # 損失値をリストに追加

        # プログレスバーに現在の損失と学習率を表示
        pbar.set_postfix({
            'train_loss': np.mean(losses[-10:]) if losses else 0, # 直近10バッチの平均損失
            'lr': optimizer.param_groups[0]['lr'] # 現在の学習率
        })

    # --- エポック終了後の処理 ---
    # 全バッチの出力とターゲットを結合
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    # AUCスコアを計算
    auc = calculate_auc(all_targets, all_outputs)
    # 平均損失を計算
    avg_loss = np.mean(losses)

    # 平均損失とAUCスコアを返す
    return avg_loss, auc


def validate(model, loader, criterion, device):
    """ 1エポック分の検証処理を行う関数 """

    model.eval() # モデルを評価モードに設定 (Dropoutなどが無効になる)
    losses = [] # 損失リスト
    all_targets = [] # ターゲットリスト
    all_outputs = [] # 出力リスト

    # 勾配計算を無効化 (メモリ節約＆計算高速化)
    with torch.no_grad():
        # 検証データローダーでループ (tqdmでプログレスバー表示)
        for batch in tqdm(loader, desc="Validation"):
            # --- バッチ処理 (train_one_epochと同様のリスト対応) ---
            if isinstance(batch['melspec'], list):
                batch_outputs = []
                batch_losses = []
                for i in range(len(batch['melspec'])):
                    inputs = batch['melspec'][i].unsqueeze(0).to(device)
                    target = batch['target'][i].unsqueeze(0).to(device)
                    output = model(inputs) # 順伝播 (評価モードなのでMixupはされない)
                    loss = criterion(output, target) # 損失計算
                    batch_outputs.append(output.detach().cpu())
                    batch_losses.append(loss.item())
                outputs = torch.cat(batch_outputs, dim=0).numpy()
                loss = np.mean(batch_losses)
                targets = batch['target'].numpy()
            # --- 通常のバッチ処理 ---
            else:
                inputs = batch['melspec'].to(device)
                targets = batch['target'].to(device)
                outputs = model(inputs) # 順伝播
                loss = criterion(outputs, targets) # 損失計算
                outputs = outputs.detach().cpu().numpy()
                targets = targets.detach().cpu().numpy()

            # 結果をリストに追加
            all_outputs.append(outputs)
            all_targets.append(targets)
            losses.append(loss if isinstance(loss, float) else loss.item())

    # --- 検証終了後の処理 ---
    # 全バッチの結果を結合
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)

    # AUCスコアを計算
    auc = calculate_auc(all_targets, all_outputs)
    # 平均損失を計算
    avg_loss = np.mean(losses)

    # 平均損失とAUCスコアを返す
    return avg_loss, auc

def calculate_auc(targets, outputs):
    """ ターゲットとモデル出力から平均クラス別AUCを計算する関数 """
    num_classes = targets.shape[1] # クラス数を取得
    aucs = [] # 各クラスのAUCを格納するリスト

    # モデル出力(ロジット)をシグモイド関数で確率に変換
    probs = 1 / (1 + np.exp(-outputs))

    # 各クラスごとにAUCを計算
    for i in range(num_classes):
        # そのクラスの陽性サンプルが存在する場合のみ計算 (存在しないとAUC計算不可)
        if np.sum(targets[:, i]) > 0:
            try:
                # sklearnのroc_auc_scoreで計算
                class_auc = roc_auc_score(targets[:, i], probs[:, i])
                aucs.append(class_auc)
            except ValueError: # AUCが計算できないケース (例:全サンプルが同じクラス)
                pass # スキップ

    # 計算できたクラスのAUCの平均値を返す (計算できなかった場合は0.0)
    return np.mean(aucs) if aucs else 0.0

## Training!

In [10]:
# === メイン学習関数 ===

def run_training(df, cfg):
    """
    交差検証 (Cross-Validation) を使ってモデル学習全体を実行する関数
    事前計算スペクトログラム利用、またはオンデマンド生成に対応
    """

    # 分類情報からクラスリストとクラス数を取得
    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
    species_ids = taxonomy_df['primary_label'].tolist()
    cfg.num_classes = len(species_ids)

    # デバッグモードが有効なら、設定を更新 (エポック数、Fold数)
    if cfg.debug:
        cfg.update_debug_settings()

    # --- スペクトログラムデータの準備 ---
    spectrograms = None # 初期化
    # 事前計算データを使用する場合 (LOAD_DATA=True)
    if cfg.LOAD_DATA:
        print("Loading pre-computed mel spectrograms from NPY file...")
        try:
            # NPYファイルをロード (allow_pickle=Trueで辞書などのオブジェクトも読める)
            spectrograms = np.load(cfg.spectrogram_npy, allow_pickle=True).item()
            print(f"Loaded {len(spectrograms)} pre-computed mel spectrograms")
        except Exception as e:
            # NPYファイルのロードに失敗した場合
            print(f"Error loading pre-computed spectrograms: {e}")
            print("Will generate spectrograms on-the-fly instead.")
            cfg.LOAD_DATA = False # オンデマンド生成モードに切り替え

    # オンデマンド生成の場合 (LOAD_DATA=False)
    if not cfg.LOAD_DATA:
        print("Will generate spectrograms on-the-fly during training.")
        # Datasetクラスでファイルパスとサンプル名が必要なので、なければ作成
        if 'filepath' not in df.columns:
            df['filepath'] = cfg.train_datadir + '/' + df.filename
        if 'samplename' not in df.columns:
            df['samplename'] = df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])

    # --- 交差検証のセットアップ ---
    # StratifiedKFold を初期化 (クラス比率を保って分割、シャッフルあり、乱数シード固定)
    skf = StratifiedKFold(n_splits=cfg.n_fold, shuffle=True, random_state=cfg.seed)

    # 各Foldのベストスコアを格納するリスト
    best_scores = []

    # --- Foldごとのループ ---
    # skf.splitで訓練データと検証データのインデックスを取得
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
        # cfg.selected_foldsに含まれないFoldはスキップ
        if fold not in cfg.selected_folds:
            continue

        print(f'\n{"="*30} Fold {fold} {"="*30}')

        # 訓練データと検証データをインデックスで抽出
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)

        print(f'Training set: {len(train_df)} samples')
        print(f'Validation set: {len(val_df)} samples')

        # --- DatasetとDataLoaderの作成 ---
        # 訓練用DatasetF
        train_dataset = BirdCLEFDatasetFromNPY(train_df, cfg, spectrograms=spectrograms, mode='train')
        # 検証用Dataset
        val_dataset = BirdCLEFDatasetFromNPY(val_df, cfg, spectrograms=spectrograms, mode='valid')

        # 訓練用DataLoader
        train_loader = DataLoader(
            train_dataset,
            batch_size=cfg.batch_size, # バッチサイズ
            shuffle=True,             # データをシャッフルする
            num_workers=cfg.num_workers,# データ読み込みの並列プロセス数
            pin_memory=True,          # GPU転送を高速化 (かもしれない)
            collate_fn=collate_fn,    # カスタムcollate関数
            drop_last=True            # バッチサイズに満たない最後のバッチを捨てる
        )

        # 検証用DataLoader
        val_loader = DataLoader(
            val_dataset,
            batch_size=cfg.batch_size,
            shuffle=False,            # 検証時はシャッフルしない
            num_workers=cfg.num_workers,
            pin_memory=True,
            collate_fn=collate_fn     # drop_last=False (デフォルト)
        )

        # --- モデル、オプティマイザ、損失関数、スケジューラの初期化 ---
        model = BirdCLEFModel(cfg).to(cfg.device) # モデルを初期化し、GPUに転送
        optimizer = get_optimizer(model, cfg) # オプティマイザ取得
        criterion = get_criterion(cfg) # 損失関数取得

        # スケジューラの初期化 (OneCycleLRはステップ数が必要なので特別扱い)
        if cfg.scheduler == 'OneCycleLR':
            scheduler = lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=cfg.lr, # 最大学習率
                steps_per_epoch=len(train_loader), # 1エポックあたりのステップ数
                epochs=cfg.epochs, # 総エポック数
                pct_start=0.1 # 学習率上昇フェーズの割合
            )
        else:
            scheduler = get_scheduler(optimizer, cfg) # 通常のスケジューラ取得

        # --- エポックごとのループ ---
        best_auc = 0 # ベストスコア初期化
        best_epoch = 0 # ベストスコア達成エポック初期化

        for epoch in range(cfg.epochs):
            print(f"\nEpoch {epoch+1}/{cfg.epochs}")

            # --- 学習フェーズ ---
            train_loss, train_auc = train_one_epoch(
                model,
                train_loader,
                optimizer,
                criterion,
                cfg.device,
                # OneCycleLRはステップ毎更新なのでここでは渡さない (train_one_epoch内で処理)
                scheduler if isinstance(scheduler, lr_scheduler.OneCycleLR) else None
            )

            # --- 検証フェーズ ---
            val_loss, val_auc = validate(model, val_loader, criterion, cfg.device)

            # --- スケジューラの更新 (エポック終了時) ---
            if scheduler is not None and not isinstance(scheduler, lr_scheduler.OneCycleLR):
                # ReduceLROnPlateau は検証ロスで判断
                if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(val_loss)
                # その他のスケジューラは単純にstep()
                else:
                    scheduler.step()

            # エポック結果の表示
            print(f"Train Loss: {train_loss:.4f}, Train AUC: {train_auc:.4f}")
            print(f"Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}")

            # --- ベストスコア更新 & モデル保存 ---
            if val_auc > best_auc: # 検証AUCが過去最高なら
                best_auc = val_auc # ベストスコア更新
                best_epoch = epoch + 1 # ベストエポック更新
                print(f"New best AUC: {best_auc:.4f} at epoch {best_epoch}")

                # モデルの状態、オプティマイザの状態、スケジューラの状態などを保存
                torch.save({
                    'model_state_dict': model.state_dict(), # モデルの重みなど
                    'optimizer_state_dict': optimizer.state_dict(), # オプティマイザの状態
                    'scheduler_state_dict': scheduler.state_dict() if scheduler else None, # スケジューラの状態
                    'epoch': epoch, # 現在のエポック
                    'val_auc': val_auc, # 検証AUC
                    'train_auc': train_auc, # 訓練AUC
                    'cfg': cfg # 使用した設定 (後で確認用)
                }, f"model_fold{fold}.pth") # ファイル名にFold番号を含める

        # Fold終了時のベストスコアをリストに追加
        best_scores.append(best_auc)
        print(f"\nBest AUC for fold {fold}: {best_auc:.4f} at epoch {best_epoch}")

        # --- Fold間のメモリ解放 ---
        del model, optimizer, scheduler, train_loader, val_loader # オブジェクト削除
        torch.cuda.empty_cache() # PyTorchのキャッシュクリア
        gc.collect() # Pythonのガベージコレクション

    # --- 全Fold終了後の結果表示 ---
    print("\n" + "="*60)
    print("Cross-Validation Results:")
    for i, score in enumerate(best_scores):
        print(f"Fold {cfg.selected_folds[i]}: {score:.4f}") # 実行したFoldのスコア表示
    print(f"Mean AUC: {np.mean(best_scores):.4f}") # 平均AUC表示
    print("="*60)

In [11]:
# === メイン実行ブロック ===
# このスクリプトが直接実行された場合にのみ実行される部分

if __name__ == "__main__":
    import time # 時間計測用 (すでにインポート済みだが念のため)

    print("\nLoading training data...")
    # 学習用メタデータCSVを読み込み
    train_df = pd.read_csv(cfg.train_csv)
    # 分類情報CSVを読み込み (run_training内でも読み込むが、ここで読んでも良い)
    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)

    print("\nStarting training...")
    # 現在のデータロード設定を表示
    print(f"LOAD_DATA is set to {cfg.LOAD_DATA}")
    if cfg.LOAD_DATA:
        print("Using pre-computed mel spectrograms from NPY file")
    else:
        print("Will generate spectrograms on-the-fly during training")

    # メインの学習関数を実行
    run_training(train_df, cfg)

    print("\nTraining complete!")

# === セルの出力 ===
# (以下は実際の実行結果のログ)
# Loading training data...
#
# Starting training...
# LOAD_DATA is set to True
# Using pre-computed mel spectrograms from NPY file
# Loading pre-computed mel spectrograms from NPY file...
# Loaded 28564 pre-computed mel spectrograms  <= NPYロード成功
#
# ============================== Fold 0 ============================== <= DebugモードなのでFold 0のみ実行
# Training set: 22851 samples
# Validation set: 5713 samples
# Found 22851 matching spectrograms for train dataset out of 22851 samples <= データ数確認
# Found 5713 matching spectrograms for valid dataset out of 5713 samples
# model.safetensors: 100% |██████████| 21.4M/21.4M [00:00<00:00, 86.8MB/s] <= timmがモデルロード
#
# Epoch 1/2  <= Debugモードなので2エポックのみ
# Training: 100% |██████████| 31/31 [00:06<00:00,  5.75it/s, train_loss=0.0344, lr=0.0005] <= 学習ループ
# Validation: 100% |██████████| 32/32 [00:02<00:00, 22.08it/s] <= 検証ループ
# Train Loss: 0.1913, Train AUC: 0.4928
# Val Loss: 0.0435, Val AUC: 0.4954 <= エポック1の結果
# New best AUC: 0.4954 at epoch 1 <= ベストスコア更新、モデル保存
#
# Epoch 2/2
# Training: 100% |██████████| 31/31 [00:05<00:00,  5.70it/s, train_loss=0.0308, lr=0.000488] <= LRが少し下がっている(CosineAnnealing)
# Validation: 100% |██████████| 32/32 [00:01<00:00, 22.00it/s]
# Train Loss: 0.0320, Train AUC: 0.4895
# Val Loss: 0.0319, Val AUC: 0.4904 <= エポック2の結果 (AUCは低下)
#
# Best AUC for fold 0: 0.4954 at epoch 1 <= Fold 0の最終ベストスコア
#
# ============================================================
# Cross-Validation Results:
# Fold 0: 0.4954
# Mean AUC: 0.4954 <= 全Fold (今回はFold 0のみ) の平均AUC
# ============================================================
#
# Training complete!


Loading training data...

Starting training...
LOAD_DATA is set to True
Using pre-computed mel spectrograms from NPY file
Loading pre-computed mel spectrograms from NPY file...
Loaded 28564 pre-computed mel spectrograms

============================== Fold 0 ==============================
Training set: 22851 samples
Validation set: 5713 samples
Found 22851 matching spectrograms for train dataset out of 22851 samples
Found 5713 matching spectrograms for valid dataset out of 5713 samples


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]


Epoch 1/10


Training:   0%|          | 0/31 [00:00<?, ?it/s]

Validation:   0%|          | 0/32 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/32 [00:00<?, ?it/s]

Train Loss: 0.0295, Train AUC: 0.5559
Val Loss: 0.0311, Val AUC: 0.5370
New best AUC: 0.5370 at epoch 9

Epoch 10/10


Training:   0%|          | 0/31 [00:00<?, ?it/s]

Validation:   0%|          | 0/32 [00:00<?, ?it/s]

Train Loss: 0.0295, Train AUC: 0.5404
Val Loss: 0.0310, Val AUC: 0.5369

Best AUC for fold 0: 0.5370 at epoch 9

Cross-Validation Results:
Fold 0: 0.5370
Mean AUC: 0.5370

Training complete!
